In [1]:
import matplotlib.pyplot as plt
import numpy as np
import helper
import time
import datetime
import torch.nn as nn
import torchvision.models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import torchvision.utils
import torch
import pandas as pd
from torchinfo import summary
from PIL import Image
from torchvision.transforms import ToTensor
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from copy import copy
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
import math
from torcheval.metrics import BinaryAccuracy
import os
import torchmetrics
import timm
import segmentation_models_pytorch as smp
import random
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
batch_size=4
image_count=50
img_size=512
tf = ToTensor()
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [16]:
test_image_list=glob('../../data/1-cycle_30%_중간데이터/segmentation/test/image/*.tiff')
test_tumor_mask_list=[f.replace('/image/', '/polygon/TP_tumor/') for f in test_image_list]
test_normal_mask_list=[f.replace('/image/', '/polygon/NT_normal/') for f in test_image_list]
train_image_list=glob('../../data/1-cycle_30%_중간데이터/segmentation/train/image/*.tiff')
train_tumor_mask_list=[f.replace('/image/', '/polygon/TP_tumor/') for f in train_image_list]
train_normal_mask_list=[f.replace('/image/', '/polygon/NT_normal/') for f in train_image_list]

class CustomDataset(Dataset):
    def __init__(self, image_list, tumor_label_list,normal_label_list,count):
        self.img_path = image_list
        self.tumor_label = tumor_label_list
        self.normal_label = normal_label_list
        self.count=count
    def __len__(self):
        return self.count

    def __getitem__(self, idx):
        a=random.randint(0,len(self.img_path)-1)
        image_path = self.img_path[a]
        tumor_label_path =  self.tumor_label[a]
        normal_label_path =  self.normal_label[a]
        image=1-tf(np.array(expand2square(Image.open(image_path),(255,255,255)).resize((img_size,img_size))))
        mask=torch.zeros((3,img_size,img_size))
        msk_tumor=np.array((expand2square(Image.open(tumor_label_path),(0,0,0)).convert('L')).resize((img_size,img_size)))
        msk_normal=np.array((expand2square(Image.open(normal_label_path),(0,0,0)).convert('L')).resize((img_size,img_size)))
        msk_back=np.where((msk_tumor+msk_normal)==0,255,0)
        mask[0]=tf(msk_back)
        mask[1]=tf(msk_tumor)
        mask[2]=tf(msk_normal)
        return image, mask

    
train_dataset = CustomDataset(train_image_list, train_tumor_mask_list,train_normal_mask_list,10641)

test_dataset = CustomDataset(test_image_list, test_tumor_mask_list,test_normal_mask_list,1330)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [13]:
len(train_dataset)

10641

In [18]:
train_image1=torch.zeros((10641,3,img_size,img_size))
train_mask1=torch.zeros((10641,3,img_size,img_size))
test_image1=torch.zeros((1330,3,img_size,img_size))
test_mask1=torch.zeros((1330,3,img_size,img_size))
for i in range(len(train_image1)):
    a=random.randint(0,len(train_image)-1)
    train_image1[i]=train_image[a]
    train_mask1[i]=train_mask[a]
for i in range(len(test_image1)):
    a=random.randint(0,len(test_image)-1)
    test_image1[i]=test_image[a]
    test_mask1[i]=test_mask[a]
    
train_dataset = CustomDataset(train_image1, train_mask1)

test_dataset = CustomDataset(test_image1, test_mask1)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

TypeError: __init__() missing 2 required positional arguments: 'normal_label_list' and 'count'

In [14]:
model=smp.UnetPlusPlus('efficientnet-b6', in_channels=3, classes=3).to(device)
summary(model,(batch_size,3,img_size,img_size))
def dice_loss(pred, target, num_classes=3):
    smooth = 1.
    dice_per_class = torch.zeros(num_classes).to(pred.device)
    
    for class_id in range(num_classes):
        pred_class = pred[:, class_id, ...]
        target_class = target[:, class_id, ...]

        intersection = torch.sum(pred_class * target_class)
        A_sum = torch.sum(pred_class * pred_class)
        B_sum = torch.sum(target_class * target_class)

        dice_per_class[class_id] = 1 - (2. * intersection + smooth) / (A_sum + B_sum + smooth)

    return torch.mean(dice_per_class)



In [17]:
train_loss_list=[]
val_loss_list=[]
train_acc_list=[]
val_acc_list=[]
MIN_loss=5000
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-4)
metrics = defaultdict(float)
for epoch in range(30):
    train=tqdm(train_dataloader)
    count=0
    running_loss = 0.0
    acc_loss=0
    for x, y in train:
        model.train()
        y = y.to(device).float()
        count+=1
        x=x.to(device).float()
        optimizer.zero_grad()  # optimizer zero 로 초기화
        predict = model(x).to(device)
        cost = dice_loss(predict, y,num_classes=3) # cost 구함
        acc=1-dice_loss(predict, y,num_classes=3)
        cost.backward() # cost에 대한 backward 구함
        optimizer.step() 
        running_loss += cost.item()
        acc_loss+=acc
        train.set_description(f"epoch: {epoch+1}/{30} Step: {count+1} dice_loss : {running_loss/count:.4f} dice_score: {acc_loss/count:.4f}")
    train_loss_list.append((running_loss/count))
    train_acc_list.append((acc_loss/count).cpu().detach().numpy())
#validation
    val=tqdm(validation_dataloader)
    model.eval()
    count=0
    val_running_loss=0.0
    acc_loss=0
    with torch.no_grad():
        for x, y in val:
            y = y.to(device).float()
            count+=1
            x=x.to(device).float()
            
            predict = model(x).to(device)
            cost = dice_loss(predict, y,num_classes=3) # cost 구함
            acc=1-dice_loss(predict, y,num_classes=3)
            val_running_loss+=cost.item()
            acc_loss+=acc
            val.set_description(f"Validation epoch: {epoch+1}/{30} Step: {count+1} dice_loss : {val_running_loss/count:.4f}  dice_score: {acc_loss/count:.4f}")
        val_loss_list.append((val_running_loss/count))
        val_acc_list.append((acc_loss/count).cpu().detach().numpy())
        
    if MIN_loss>(val_running_loss/count):
        torch.save(model.state_dict(), '../../model/NestedUNet_3class_callback.pt')
        MIN_loss=(val_running_loss/count)
        
    if epoch%20==5:
        plt.figure(figsize=(10,5))
        plt.subplot(1, 2, 1) 
        plt.title('loss_graph')
        plt.plot(np.arange(epoch+1),train_loss_list,label='train_loss')
        plt.plot(np.arange(epoch+1),val_loss_list,label='validation_loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.ylim([0, 1]) 
        plt.legend()
        plt.subplot(1, 2, 2)  
        plt.title('acc_graph')
        plt.plot(np.arange(epoch+1),train_acc_list,label='train_acc')
        plt.plot(np.arange(epoch+1),val_acc_list,label='validation_acc')
        plt.xlabel('epoch')
        plt.ylabel('accuracy')
        plt.ylim([0, 1]) 
        plt.legend()
        plt.show()
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1) 
plt.title('loss_graph')
plt.plot(np.arange(epoch+1),train_loss_list,label='train_loss')
plt.plot(np.arange(epoch+1),val_loss_list,label='validation_loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.ylim([0, 1]) 
plt.legend()
plt.subplot(1, 2, 2)  
plt.title('acc_graph')
plt.plot(np.arange(epoch+1),train_acc_list,label='train_acc')
plt.plot(np.arange(epoch+1),val_acc_list,label='validation_acc')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.ylim([0, 1]) 
plt.legend()
plt.show()

  0%|          | 0/2660 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
predict.shape

torch.Size([8, 3, 512, 512])